In [1]:
import json

with open(r"C:\Users\Vishwas\Downloads\iotdata_vishnusir.json", 'r') as file:
    data = json.load(file)

In [4]:
data

{"select * from tbl_customer_iot_data tcid where customer_id='261cb253-f11a-461c-9dc0-b608549c4454'": [{'customer_iot_id': '4069269e-94af-4e7e-9c35-a5be213148fb',
   'customer_id': '261cb253-f11a-461c-9dc0-b608549c4454',
   'data_origin': None,
   'device_type': None,
   'device_id': None,
   'start_date_time': '2025-03-22T08:00:00.000Z',
   'end_date_time': '2025-03-22T08:00:00.000Z',
   'iot_type': 'HeartRate',
   'iot_unit': 'count/min',
   'reading_value': 71,
   'created_at': '2025-03-26T04:05:59.988Z',
   'updated_at': None,
   'created_by': '4d8f9de8-edd3-42bb-9ef1-7f91e8607956',
   'updated_by': None,
   'manual_type': 'HEARTRATE'},
  {'customer_iot_id': 'a48d1c33-e2f1-4dab-8d40-73492c5d637f',
   'customer_id': '261cb253-f11a-461c-9dc0-b608549c4454',
   'data_origin': None,
   'device_type': None,
   'device_id': None,
   'start_date_time': '2025-03-21T08:00:00.000Z',
   'end_date_time': '2025-03-21T08:00:00.000Z',
   'iot_type': 'HeartRate',
   'iot_unit': 'count/min',
   'rea

In [7]:
iot_type = set()
for sample in data["select * from tbl_customer_iot_data tcid where customer_id='261cb253-f11a-461c-9dc0-b608549c4454'"]:
      iot_type.add(sample['iot_type'])

In [8]:
iot_type

{'AsleepDeep',
 'FlightsClimbed',
 'HeartRate',
 'OxygenSaturation',
 'StepCount',
 'TotalCaloriesBurned',
 'diastolicBp',
 'systolicBp'}

extraction and data prepration process

In [61]:
import json
from dateutil.parser import parse as parse_datetime
from datetime import datetime, timedelta
import math

from datetime import datetime

def parse_meal_time(meal_time_str):
    return datetime.strptime(meal_time_str, "%Y-%m-%d %I:%M %p")

# Function to check if meal is within the 15-minute interval [T-15min, T)
def is_within_time_interval(cgm_time, meal_time):
    interval_start = cgm_time - timedelta(minutes=15)
    return interval_start <= meal_time < cgm_time


In [62]:
with open(r"C:\Users\Vishwas\Downloads\eda752c3-ac69-4752-bce6-f718805822bf_MEAL.json") as f:
    food_data = json.load(f)
    # food_data = food_data["select * from tbl_customer_iot_data tcid where customer_id='261cb253-f11a-461c-9dc0-b608549c4454'"]

with open(r"C:\Users\Vishwas\Downloads\261cb253-f11a-461c-9dc0-b608549c4454_CGM.json") as f:
    cgm_data = json.load(f)
    cgm_data = cgm_data["select COALESCE(ROUND(glucose_reading * 18.08, 2), 0) AS glucose_reading, glucose_uom, cgm_device_id, glucose_warning, reading_date_time AT TIME ZONE 'UTC' AT TIME ZONE 'Asia/Kolkata' AS reading_date_time\r\n                    from tbl_customer_cgm_data\r\n                    where customer_id = '261cb253-f11a-461c-9dc0-b608549c4454'\r\n                    order by reading_date_time desc"]

In [63]:
def is_within_time_interval(cgm_time, meal_time):
    interval_start = cgm_time - timedelta(minutes=15)
    return interval_start <= meal_time < cgm_time

# Process CGM and Meal data
result = []
for cgm in cgm_data:
    cgm_time = datetime.strptime(cgm["reading_date_time"], "%Y-%m-%dT%H:%M:%S.000Z")
    meal_info = []

    for meal in food_data:
        meal_time = parse_meal_time(meal["mealTime"])
        if is_within_time_interval(cgm_time, meal_time):
            # Extract nutrition data if available
            nutrition = {}
            if "food" in meal and "nutrition" in meal["food"] and "macro" in meal["food"]["nutrition"]:
                macro = meal["food"]["nutrition"]["macro"]
                nutrition = {
                    "protein": f"{macro.get('protein', {}).get('value', 0)} g",
                    "energy": f"{macro.get('energy', {}).get('value', 0)} kcal",
                    "total_dietary_fibre": f"{macro.get('total_dietary_fibre', {}).get('value', 0)} g",
                    "carbohydrates": f"{macro.get('carbohydrates', {}).get('value', 0)} g"
                }
            meal_info.append({
                "mealType": meal["mealType"],
                "mealTime": meal["mealTime"],
                "nutrition": nutrition if nutrition else {}
            })

    # Create output entry
    entry = {
        "glucose_reading": cgm["glucose_reading"],
        "reading_date_time": cgm["reading_date_time"],
        "meal_info": meal_info if meal_info else None
    }
    result.append(entry)

ValueError: time data '2025-03-06 22:15 PM' does not match format '%Y-%m-%d %I:%M %p'

In [67]:
def ceil_to_next_15(dt):
    # Round to the next 15-minute mark
    dt = dt.replace(second=0, microsecond=0)
    dt = dt + timedelta(minutes=(15 - dt.minute % 15) % 15)
    # Format the datetime to the desired format
    return dt.strftime("%Y-%m-%dT%H:%M:%S.000Z")

dt1 = datetime.strptime("2025-03-11 10:01 AM", "%Y-%m-%d %I:%M %p")

ceil_to_next_15(dt1)

'2025-03-11T10:15:00.000Z'

In [55]:
def extract_relevant_nutrition(food_entry):
    print(food_entry)
    nutrition = food_entry["food"]["nutrition"]["macro"]
    return {
        "mealType": food_entry["mealType"],
        "mealTime": parse_datetime(food_entry["mealTime"].replace("AM", "").replace("PM", "").strip()),
        "carbohydrates": nutrition.get("carbohydrates", {}),
        "total_dietary_fibre": nutrition.get("total_dietary_fibre", {}),
        "energy": nutrition.get("energy", {}),
        "protein": nutrition.get("protein", {})
    }

In [56]:
def get_nested(d, path):
    for key in path:
        if isinstance(d, dict):
            d = d.get(key)
        else:
            return None
    return d

In [57]:
def merge_cgm_and_meals(cgm_data, food_data):
    # Create a map from rounded meal time to list of meals
    meals_by_time = {}
    for meal in food_data:
        meal_time = parse_meal_time(meal["mealTime"])
        rounded_time = round_up_to_next_15min(meal_time)
        meal_info = {
            "mealType": meal["mealType"],
            "mealTime": meal["mealTime"],
            "nutrition": {
                "protein": get_nested(meal, ["food", "nutrition", "macro", "protein", "value"]),
                "energy": get_nested(meal, ["food", "nutrition", "macro", "energy", "value"]),
                "total_dietary_fibre": get_nested(meal, ["food", "nutrition", "macro", "total_dietary_fibre", "value"]),
                "carbohydrates": get_nested(meal, ["food", "nutrition", "macro", "carbohydrates", "value"])
            }
        }
        meals_by_time.setdefault(rounded_time, []).append(meal_info)

    # Merge CGM data with corresponding meals
    merged_output = []
    for entry in cgm_data:
        cgm_time = parse_datetime(entry["reading_date_time"]).replace(second=0, microsecond=0)
        meal_info = meals_by_time.get(cgm_time, [])
        merged_output.append({
            "glucose_reading": entry["glucose_reading"],
            "reading_date_time": entry["reading_date_time"],
            "meal_info": meal_info  # could be empty list if no meals
        })

    return merged_output

In [59]:
matched = merge_cgm_and_meals(cgm_data, food_data)


In [60]:
matched

[{'glucose_reading': 164.53,
  'reading_date_time': '2025-03-11T18:15:00.000Z',
  'meal_info': []},
 {'glucose_reading': 162.72,
  'reading_date_time': '2025-03-11T18:00:00.000Z',
  'meal_info': []},
 {'glucose_reading': 160.91,
  'reading_date_time': '2025-03-11T17:45:00.000Z',
  'meal_info': []},
 {'glucose_reading': 159.1,
  'reading_date_time': '2025-03-11T17:30:00.000Z',
  'meal_info': []},
 {'glucose_reading': 162.72,
  'reading_date_time': '2025-03-11T17:15:00.000Z',
  'meal_info': []},
 {'glucose_reading': 84.98,
  'reading_date_time': '2025-03-11T17:00:00.000Z',
  'meal_info': []},
 {'glucose_reading': 81.36,
  'reading_date_time': '2025-03-11T16:45:00.000Z',
  'meal_info': []},
 {'glucose_reading': 81.36,
  'reading_date_time': '2025-03-11T16:30:00.000Z',
  'meal_info': []},
 {'glucose_reading': 54.24,
  'reading_date_time': '2025-03-11T16:15:00.000Z',
  'meal_info': []},
 {'glucose_reading': 54.24,
  'reading_date_time': '2025-03-11T16:00:00.000Z',
  'meal_info': []},
 {'glu